In [1]:
import numpy as np
import simple_dmrg.dmrg_functions as dmrg
import simple_dmrg.mps_functions as mps_func
import simple_dmrg.mpo_operations as mpo_ops
import simple_dmrg.mpo_construction as make_mpo
import scipy.sparse as sps
import time
from pathlib import Path

In [2]:
# phys dim 4, each site has 4 states: |0>, |u>, |d>, |2>
# |0> is the vacuum state
# u for spin up, d for spin down, |2> := a_u_dag.a_d_dag|0> for both up and down
a_u_2darray = np.array([[0, 1, 0, 0],
                        [0, 0, 0, 0],
                        [0, 0, 0, 1],
                        [0, 0, 0, 0]])
a_d_2darray = np.array([[0, 0, 1, 0],
                        [0, 0, 0, -1],
                        [0, 0, 0, 0],
                        [0, 0, 0, 0]])
a_u = sps.csr_matrix(a_u_2darray)
a_d = sps.csr_matrix(a_d_2darray)
a_u_dag = a_u.transpose().conj()
a_d_dag = a_d.transpose().conj()
n_u = a_u_dag @ a_u
n_d = a_d_dag @ a_d
n_part = n_u + n_d

print(f"a_u = \n{a_u.toarray()}")
print(f"a_d = \n{a_d.toarray()}")
print(f"a_u_dag = \n{a_u_dag.toarray()}")
print(f"a_d_dag = \n{a_d_dag.toarray()}")
print(f"n_u = \n{n_u.toarray()}")
print(f"n_d = \n{n_d.toarray()}")
print(f"n_part = \n{n_part.toarray()}")

a_u = 
[[0 1 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 0 0]]
a_d = 
[[ 0  0  1  0]
 [ 0  0  0 -1]
 [ 0  0  0  0]
 [ 0  0  0  0]]
a_u_dag = 
[[0 0 0 0]
 [1 0 0 0]
 [0 0 0 0]
 [0 0 1 0]]
a_d_dag = 
[[ 0  0  0  0]
 [ 0  0  0  0]
 [ 1  0  0  0]
 [ 0 -1  0  0]]
n_u = 
[[0 0 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 1]]
n_d = 
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 1]]
n_part = 
[[0 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 2]]


In [3]:
def build_fermionic_operators(num_sites):
    a_u_list = []
    a_d_list = []
    a_u_dag_list = []
    a_d_dag_list = []
    id = sps.eye(4, format='csr')
    parity_2darray = np.array([[1, 0, 0, 0],
                               [0, -1, 0, 0],
                               [0, 0, -1, 0],
                               [0, 0, 0, 1]])
    parity = sps.csr_matrix(parity_2darray)
    for i in range(num_sites):
        for j in range(num_sites):
            if j == 0:
                if i == 0:
                    a_u_local = a_u
                    a_d_local = a_d
                else:
                    a_u_local = parity
                    a_d_local = parity
            elif j < i:
                a_u_local = sps.kron(a_u_local, parity)
                a_d_local = sps.kron(a_d_local, parity)
            elif j == i:
                a_u_local = sps.kron(a_u_local, a_u)
                a_d_local = sps.kron(a_d_local, a_d)
            else:
                a_u_local = sps.kron(a_u_local, id)
                a_d_local = sps.kron(a_d_local, id)
        a_u_list.append(a_u_local)
        a_d_list.append(a_d_local)
        a_u_dag_list.append(a_u_local.transpose())
        a_d_dag_list.append(a_d_local.transpose())
    

    return a_u_list, a_d_list, a_u_dag_list, a_d_dag_list

def build_number_operators(a_u_list, a_d_list, a_u_dag_list, a_d_dag_list):
    num_sites = len(a_u_list)
    n_u_list = []
    n_d_list = []
    n_part_list = []

    for i in range(num_sites):
        n_u_list.append(a_u_dag_list[i] @ a_u_list[i])
        n_d_list.append(a_d_dag_list[i] @ a_d_list[i])
        n_part_list.append(n_u_list[i] + n_d_list[i])

    # Build total number operator
    n_total_op = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for iiter in range(num_sites):
        n_total_op += n_part_list[iiter]

    return n_u_list, n_d_list, n_part_list, n_total_op

# np.set_printoptions(threshold=np.inf)
# print(f"a_u_list[0] = \n{a_u_list[0].toarray()}")
# print(f"a_u_list[1] = \n{a_u_list[1].toarray()}")
# print(f"a_u_list[2] = \n{a_u_list[2].toarray()}")
# print(f"a_d_list[0] = \n{a_d_list[0].toarray()}")
# print(f"a_d_list[1] = \n{a_d_list[1].toarray()}")
# print(f"a_d_list[2] = \n{a_d_list[2].toarray()}")
# print(f"a_u_dag_list[0] = \n{a_u_dag_list[0].toarray()}")
# print(f"a_u_dag_list[1] = \n{a_u_dag_list[1].toarray()}")
# print(f"a_u_dag_list[2] = \n{a_u_dag_list[2].toarray()}")
# print(f"a_d_dag_list[0] = \n{a_d_dag_list[0].toarray()}")
# print(f"a_d_dag_list[1] = \n{a_d_dag_list[1].toarray()}")
# print(f"a_d_dag_list[2] = \n{a_d_dag_list[2].toarray()}")



In [4]:
def check_all_anti_commutation_relations(a_u,a_d,a_u_dag,a_d_dag,a_u_list,a_d_list,a_u_dag_list,a_d_dag_list):
    num_sites = len(a_u_list)

    assert (a_u @ a_u_dag + a_u_dag @ a_u-sps.eye(4)).nnz ==0
    assert (a_d @ a_d_dag + a_d_dag @ a_d-sps.eye(4)).nnz ==0

    assert (a_u @ a_d_dag + a_d_dag @ a_u).nnz == 0
    assert (a_d @ a_u_dag + a_u_dag @ a_d).nnz == 0

    assert (a_u @ a_d + a_d @ a_u).nnz == 0
    assert (a_u_dag @ a_d_dag + a_d_dag @ a_u_dag).nnz == 0
    
    for i in range(num_sites):
        for j in range(num_sites):
            
            
            if i == j:
                assert (a_u_list[i] @ a_u_dag_list[j] + a_u_dag_list[j] @ a_u_list[i]-sps.eye(4**num_sites)).nnz ==0
                assert (a_d_list[i] @ a_d_dag_list[j] + a_d_dag_list[j] @ a_d_list[i]-sps.eye(4**num_sites)).nnz ==0
            else:
                assert (a_u_list[i] @ a_u_dag_list[j] + a_u_dag_list[j] @ a_u_list[i]).nnz == 0
                assert (a_d_list[i] @ a_d_dag_list[j] + a_d_dag_list[j] @ a_d_list[i]).nnz == 0

            assert (a_u_list[i] @ a_d_dag_list[j] + a_d_dag_list[j] @ a_u_list[i]).nnz == 0
            assert (a_d_list[i] @ a_u_dag_list[j] + a_u_dag_list[j] @ a_d_list[i]).nnz == 0

            assert (a_u_list[i] @ a_d_list[j] + a_d_list[j] @ a_u_list[i]).nnz == 0
            assert (a_u_dag_list[i] @ a_d_dag_list[j] + a_d_dag_list[j] @ a_u_dag_list[i]).nnz == 0
                
    # print("All anti-commutation relations are satisfied.")

In [5]:
def build_hubbard_hamiltonian(
    num_sites, hopping_strength, onsite_interaction, chemical_potential
):
    a_u_list, a_d_list, a_u_dag_list, a_d_dag_list = build_fermionic_operators(
        num_sites
    )
    check_all_anti_commutation_relations(
        a_u, a_d, a_u_dag, a_d_dag, a_u_list, a_d_list, a_u_dag_list, a_d_dag_list
    )
    n_u_list, n_d_list, n_part_list, n_total_op = build_number_operators(
        a_u_list, a_d_list, a_u_dag_list, a_d_dag_list
    )
    # Build Hubbard Hamiltonian with open boundary conditions
    # Following Eqn. 11 of https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.cond-mat.de/events/correl16/manuscripts/scalettar.pdf&ved=2ahUKEwjvus-1h4yHAxXIJzQIHdXNO1UQFnoECBAQAw&usg=AOvVaw1RrwrDGqwV43PYpA8JlhDW
    t = hopping_strength
    U = onsite_interaction
    mu = chemical_potential
    hopping_term = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    interaction_term = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for iiter in range(num_sites - 1):
        hopping_term += -1 * t * (a_u_dag_list[iiter] @ a_u_list[iiter + 1])
        hopping_term += -1 * t * (a_d_dag_list[iiter] @ a_d_list[iiter + 1])
        hopping_term += -1 * t * (a_u_dag_list[iiter + 1] @ a_u_list[iiter])
        hopping_term += -1 * t * (a_d_dag_list[iiter + 1] @ a_d_list[iiter])
        # print(iiter)

    # #Periodic boundary conditions
    # for iiter in range(num_sites):
    #     hopping_term += -1 * t * (a_u_dag_list[iiter] @ a_u_list[(iiter + 1) % num_sites])
    #     hopping_term += -1 * t * (a_d_dag_list[iiter] @ a_d_list[(iiter + 1) % num_sites])
    #     hopping_term += -1 * t * (a_u_dag_list[(iiter + 1) % num_sites] @ a_u_list[iiter])
    #     hopping_term += -1 * t * (a_d_dag_list[(iiter + 1) % num_sites] @ a_d_list[iiter])

    # # Onsite interaction term
    # for iiter in range(num_sites):
    #     interaction_term += U * (n_u_list[iiter] @ n_d_list[iiter])

    # # Chemical potential, adjusted for half-filling
    # chemical_potential_term = -1 * (mu + (U / 2)) * n_total_op

    # # Constant Shift
    # constant_shift = 2*sps.eye(4**num_sites) * (U / 4)

    # hami_one_dim_hubbard = (
    #     hopping_term + interaction_term + chemical_potential_term + constant_shift
    # )

    # Onsite interaction term
    for iiter in range(num_sites):
        interaction_term += U * (
            (n_u_list[iiter] - 0.5 * sps.eye(4**num_sites))
            @ (n_d_list[iiter] - 0.5 * sps.eye(4**num_sites))
        )

    # Chemical potential, adjusted for half-filling
    chemical_potential_term = -1 * (mu) * n_total_op

    hami_one_dim_hubbard = (
        hopping_term + interaction_term + chemical_potential_term  # + constant_shift
    )

    np.set_printoptions(threshold=np.inf, linewidth=np.inf)
    # print(f"hopping_term = \n{hopping_term.toarray()}")
    # print(f"interaction_term = \n{interaction_term.toarray()}")
    # print(f"hami_one_dim_hubbard = \n{hami_one_dim_hubbard.toarray()}")
    # Check that the Hamiltonian is Hermitian
    assert (hami_one_dim_hubbard - hami_one_dim_hubbard.transpose().conj()).nnz == 0
    return (
        hami_one_dim_hubbard,
        a_u_list,
        a_d_list,
        a_u_dag_list,
        a_d_dag_list,
        n_u_list,
        n_d_list,
        n_part_list,
        n_total_op,
    )


def diagonalize_hubbard_hami(hami_one_dim_hubbard, num_states=2):
    eigvals, eigvecs = sps.linalg.eigsh(hami_one_dim_hubbard, k=num_states, which="SA")

    # eigvals += U/4 # Adjust for particle hole symmetric form of the Hamiltonian
    # Set near-zero values to zero
    eigvec = eigvecs[:, 0]
    eigvec[abs(eigvec) < 1e-10] = 0
    eigvec = eigvec / np.linalg.norm(eigvec)
    eigvec_sparse = sps.csr_array(eigvec)
    return eigvals, eigvec_sparse


def get_observables(
    eigvals,
    eigvec_sparse,
    n_total_op,
    n_u_list,
    n_d_list,
    a_u_dag_list,
    a_d_list,
    a_d_dag_list,
    a_u_list,
):
    # print(f"Ground state energy from exact diagonalization = {eigvals[0]}")
    # print(f"Ground state from exact diagonalization = {eigvec_sparse}")
    # print(f"num non-zero elements = {eigvec_sparse.nnz}")
    measured_num_particles = (
        eigvec_sparse @ n_total_op @ eigvec_sparse.conj().transpose()
    )
    measured_num_particles = measured_num_particles[0, 0]
    # print(f"Total number of particles = {measured_num_particles}")
    # Get total s_z operator
    s_z = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for iiter in range(num_sites):
        s_z += 0.5 * (n_u_list[iiter] - n_d_list[iiter])

    measured_s_z = eigvec_sparse @ s_z @ eigvec_sparse.conj().transpose()
    measured_s_z = measured_s_z[0, 0]
    # print(f"Total <Sz> = {measured_s_z}")

    # Get total s^2 operator
    s_plus = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    s_minus = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for iiter in range(num_sites):
        s_plus += a_u_dag_list[iiter] @ a_d_list[iiter]
        s_minus += a_d_dag_list[iiter] @ a_u_list[iiter]

    s_squared = s_minus @ s_plus + s_z @ s_z + s_z
    measured_s_squared = eigvec_sparse @ s_squared @ eigvec_sparse.conj().transpose()
    measured_s_squared = measured_s_squared[0, 0]
    S_val = 0.5 * (-1 + np.sqrt(1 + 4 * measured_s_squared))
    # print(f"Total <S^2> = {measured_s_squared}")
    # print(f"Total S = {S_val}")

    # Get local moment
    m_squared_op = (2 * s_z) @ (2 * s_z)
    m_squared = eigvec_sparse @ m_squared_op @ eigvec_sparse.conj().transpose()

    m_squared = m_squared[0, 0]
    # print(f"Local moment squared <m^2> = {m_squared}")
    # print(f"Eigenvalues = {eigvals}")
    return measured_num_particles, measured_s_z, measured_s_squared, S_val, m_squared

def oned_hubbard_get_max_bd(
    num_sites, hopping_strength, onsite_interaction, chemical_potential
):
    ham_build_start = time.time()

    (
        hami_one_dim_hubbard,
        a_u_list,
        a_d_list,
        a_u_dag_list,
        a_d_dag_list,
        n_u_list,
        n_d_list,
        n_part_list,
        n_total_op,
    ) = build_hubbard_hamiltonian(
        num_sites, hopping_strength, onsite_interaction, chemical_potential
    )
    ham_build_calc_time = time.time() - ham_build_start

    # Get ground state energy and wavefunction from exact sparse diagonalization

    diag_start_time = time.time()

    eigvals, eigvec_sparse = diagonalize_hubbard_hami(hami_one_dim_hubbard)

    diag_calc_time = time.time() - diag_start_time
    # print(f"diag_calc_time = {diag_calc_time}")
    measured_num_particles, measured_s_z, measured_s_squared, S_val, m_squared = (
        get_observables(
            eigvals,
            eigvec_sparse,
            n_total_op,
            n_u_list,
            n_d_list,
            a_u_dag_list,
            a_d_list,
            a_d_dag_list,
            a_u_list,
        )
    )

    mps_start_time = time.time()
    mps = mps_func.statevector_to_mps(
        state_vector=eigvec_sparse.toarray()[0],
        num_sites=num_sites,
        physical_dim=4,
        right_normalize=True,
        orig_bond_dim=None,
        verbosity=0,
    )
    mps_build_time = time.time() - mps_start_time
    # print(f"mps_build_time = {mps_build_time}")
    max_bond_dimension = 0
    for local_tensor in mps:
        max_bond_dimension = max(max_bond_dimension, local_tensor.shape[0])
        max_bond_dimension = max(max_bond_dimension, local_tensor.shape[2])
    # print(f"max_bond_dimension = {max_bond_dimension}")
    # print(f"Hamiltonian build calculation time = {ham_build_calc_time} seconds")
    rebuilt_eigenvec = mps_func.mps_to_statevector(mps=mps, verbosity = 0)
    assert np.allclose(np.abs(rebuilt_eigenvec@eigvec_sparse.toarray()[0]),1)
    print(f"hami_build_time, diag_time, mps_build_time = {ham_build_calc_time, diag_calc_time, mps_build_time}")
    return (
        ham_build_calc_time,
        eigvals,
        diag_calc_time,
        measured_num_particles,
        measured_s_z,
        measured_s_squared,
        S_val,
        m_squared,
        mps_build_time,
        max_bond_dimension,
        eigvec_sparse
    )

In [6]:
# Check the anti-commutation relations
# assert (((a_u_list[0] @ a_u_dag_list[0] + a_u_dag_list[0] @ a_u_list[0])-sps.eye(4**num_sites)).nnz) == 0
# assert (((a_u_list[1] @ a_u_dag_list[1] + a_u_dag_list[1] @ a_u_list[1])-sps.eye(4**num_sites)).nnz) ==0
# assert (((a_d_list[0] @ a_d_dag_list[0] + a_d_dag_list[0] @ a_d_list[0])-sps.eye(4**num_sites)).nnz) ==0
# assert (((a_d_list[1] @ a_d_dag_list[1] + a_d_dag_list[1] @ a_d_list[1])-sps.eye(4**num_sites)).nnz) ==0
# assert ((a_u @ a_d+ a_d @ a_u).nnz) ==0
# assert (((a_u_list[0] @ a_d_list[0] + a_d_list[0] @ a_u_list[0])).nnz) ==0
# assert (((a_u_list[1] @ a_d_list[1] + a_d_list[1] @ a_u_list[1])).nnz) ==0
# assert (((a_u_list[0] @ a_d_list[1] + a_d_list[1] @ a_u_list[0])).nnz) ==0
# assert (((a_u_list[1] @ a_d_list[0] + a_d_list[0] @ a_u_list[1])).nnz) ==0
# assert (((a_u_dag @ a_d_dag + a_d_dag @ a_u_dag).nnz)) ==0
# assert (((a_u_dag_list[0] @ a_d_dag_list[0] + a_d_dag_list[0] @ a_u_dag_list[0])).nnz) ==0
# assert (((a_u_dag_list[1] @ a_d_dag_list[1] + a_d_dag_list[1] @ a_u_dag_list[1])).nnz) ==0
# assert (((a_u_dag_list[0] @ a_d_dag_list[1] + a_d_dag_list[1] @ a_u_dag_list[0])).nnz) ==0
# assert (((a_u_dag_list[1] @ a_d_dag_list[0] + a_d_dag_list[0] @ a_u_dag_list[1])).nnz) ==0
# assert (((a_u @ a_d_dag + a_d_dag @ a_u).nnz)) ==0
# assert (((a_u_list[0] @ a_d_dag_list[0] + a_d_dag_list[0] @ a_u_list[0]).nnz)) ==0
# assert (((a_u_list[1] @ a_d_dag_list[1] + a_d_dag_list[1] @ a_u_list[1]).nnz)) ==0
# assert (((a_u_list[0] @ a_d_dag_list[1] + a_d_dag_list[1] @ a_u_list[0]).nnz)) ==0
# assert (((a_u_list[1] @ a_d_dag_list[0] + a_d_dag_list[0] @ a_u_list[1]).nnz)) ==0
# assert (((a_u_dag @ a_d + a_d @ a_u_dag).nnz)) ==0
# assert (((a_u_dag_list[0] @ a_d_list[0] + a_d_list[0] @ a_u_dag_list[0]).nnz)) ==0
# assert (((a_u_dag_list[1] @ a_d_list[1] + a_d_list[1] @ a_u_dag_list[1]).nnz)) ==0
# assert (((a_u_dag_list[0] @ a_d_list[1] + a_d_list[1] @ a_u_dag_list[0]).nnz)) ==0
# assert (((a_u_dag_list[1] @ a_d_list[0] + a_d_list[0] @ a_u_dag_list[1]).nnz)) ==0





In [7]:
num_sites_list = [3, 4, 5]
chemical_potential_list = [0.0, 0.5, -0.5]
hopping_strength_list = [1.0]
onsite_interaction_list = [-10,-4,-2,-1,0,1,2,4,10]

num_calculations = len(num_sites_list)*len(chemical_potential_list)*len(hopping_strength_list)*len(onsite_interaction_list)
count = 0
for num_sites in num_sites_list:
    for chemical_potential in chemical_potential_list:
        for hopping_strength in hopping_strength_list:
            for onsite_interaction in onsite_interaction_list:
                print(f"Calculation {count+1} of {num_calculations}")
                count += 1
                print(f"num_sites, hopping_strength, onsite_interaction, chemical_potential = {num_sites}, {hopping_strength}, {onsite_interaction}, {chemical_potential}")
# num_sites = 3
# hopping_strength = 1
# onsite_interaction = 2
# chemical_potential = 0.0







                (
                    ham_build_calc_time,
                    eigvals,
                    diag_calc_time,
                    measured_num_particles,
                    measured_s_z,
                    measured_s_squared,
                    S_val,
                    m_squared,
                    mps_build_time,
                    max_bond_dimension,
                    eigvec_sparse
                ) = oned_hubbard_get_max_bd(
                    num_sites, hopping_strength, onsite_interaction, chemical_potential
                )

                # Append data to csv
                
                csv_path = Path("1d_hubbard_data.csv")
                header_string = ("num_sites,"
                        +"hopping_strength,"
                        +"onsite_interaction,"
                        +"chemical_potential,"
                        +"ground_state_energy_ED,"
                        +"num_csf_in_ground_state,"
                        +"total_num_particles,"
                        +"total_sz,"
                        +"total_s_squared,"
                        +"total_s,"
                        +"local_moment_squared,"
                        +"first_excited_state_energy_ED,"
                        +"max_bond_dimension,"
                        +"hamiltonian_build_time,"
                        +"diagonalization_time,"
                        +"mps_build_time"
                        +"\n")
                if not csv_path.exists():
                    with open(csv_path, "w") as f:
                        f.write(header_string)
                string_to_append = (f"{num_sites:d},"
                                    f"{hopping_strength:.16f},"
                                    f"{onsite_interaction:.16f},"
                                    f"{chemical_potential:.16f},"
                                    f"{eigvals[0]:.16f},"
                                    f"{eigvec_sparse.nnz:d},"
                                    f"{measured_num_particles:.16f},"
                                    f"{measured_s_z:.16f},"
                                    f"{measured_s_squared:.16f},"
                                    f"{S_val:.16f},"
                                    f"{m_squared:.16f},"
                                    f"{eigvals[1]:.16f},"
                                    f"{max_bond_dimension:d},"
                                    f"{ham_build_calc_time:.16f},"
                                    f"{diag_calc_time:.16f},"
                                    f"{mps_build_time:.16f}"
                                    +"\n")    
                with open(csv_path, "a") as f:
                    f.write(string_to_append)
                print(string_to_append)
        

Calculation 1 of 81
num_sites, hopping_strength, onsite_interaction, chemical_potential = 3, 1.0, -10, 0.0
hami_build_time, diag_time, mps_build_time = (0.050269365310668945, 0.0016362667083740234, 0.0007345676422119141)
3,1.0000000000000000,-10.0000000000000000,0.0000000000000000,-8.0775626616949783,18,2.0984166633166019,0.0000000000000000,0.0000000000000000,0.0000000000000000,0.0000000000000000,-8.0775626616949729,4,0.0502693653106689,0.0016362667083740,0.0007345676422119

Calculation 2 of 81
num_sites, hopping_strength, onsite_interaction, chemical_potential = 3, 1.0, -4, 0.0
hami_build_time, diag_time, mps_build_time = (0.04661917686462402, 0.001651763916015625, 0.0004031658172607422)
3,1.0000000000000000,-4.0000000000000000,0.0000000000000000,-4.2360679774997889,18,3.9348201135996312,0.0000000000000000,0.0000000000000000,0.0000000000000000,0.0000000000000000,-4.2360679774997854,4,0.0466191768646240,0.0016517639160156,0.0004031658172607

Calculation 3 of 81
num_sites, hopping_stren

In [8]:


# eigvec = eigvecs[:, 0]
# eigvec.shape
# abs(eigvec).shape
# abs(eigvec) > 1e-10
# eigvec[abs(eigvec) > 1e-10]
# eigvec_trimmed = eigvec[abs(eigvec) > 1e-10]
# eigvec_sparse = sps.csr_matrix(eigvec_trimmed[:, 0])

In [9]:
# np.sqrt(4*t*t + U*U/4)

In [10]:
# for i in range(num_sites):
#     k_i = 2*np.pi/num_sites*(i+1)
#     # print(f"k_{i+1} = {k_i}")
#     # print(f"cos(k_{i+1}) = {np.cos(k_i)}")
#     print(f"-2t*cos(k_{i+1}) = {-2*t*np.cos(k_i)}")
# k_1 = 2*np.pi/num_sites
# k_2 = 4*np.pi/num_sites
# k_3 = 6*np.pi/num_sites
# k_4 = 8*np.pi/num_sites
# k_5 = 10*np.pi/num_sites

# print([np.cos(k_1),
# np.cos(k_2),
# np.
# 2*(-2*t*np.cos(k_1)-4*t*np.cos(k_2))
# -4+2*(-0.6180339887498945)

In [11]:
# for local_tensor in mps:
#     print(local_tensor.shape)

In [12]:
# 12 sites: BD = 64, At least two degenerate eigenstates (-12, -12), state vec reproduced from MPS
# (1, 4, 2)
# (2, 4, 4)
# (4, 4, 8)
# (8, 4, 16)
# (16, 4, 32)
# (32, 4, 64)
# (64, 4, 32)
# (32, 4, 16)
# (16, 4, 8)
# (8, 4, 4)
# (4, 4, 2)
# (2, 4, 1)
# 4^12 = 16,777,216, log10(16,777,216) = 7.225
#Building Hami takes about 15 min, diagonalizing (bottom two states) takes about 1 min
# RAM use about 8GB?
# hopping_strength = 0
# onsite_interaction = 4
# chemical_potential = 0.0
# Ground state energy from exact diagonalization = -12.000000000000021
# Ground state from exact diagonalization =   (0, 5592405)	0.008768326913710073
#   (0, 5592406)	0.00821675034171448
#   (0, 5592409)	-0.0054728696537591085
#   (0, 5592410)	0.004000263467219952
#   (0, 5592421)	-0.005242978873662419
#   (0, 5592422)	-0.010574532197638
#   (0, 5592425)	0.019762450567604745
#   (0, 5592426)	-0.05095303531533082
#   (0, 5592469)	0.009395291579145454
#   (0, 5592470)	-0.03443832693776153
#   (0, 5592473)	0.015556042301705271
#   (0, 5592474)	0.026920623471817146
#   (0, 5592485)	0.013334576519308297
#   (0, 5592486)	-0.01426205641383977
#   (0, 5592489)	-0.003140200791953928
#   (0, 5592490)	0.003783184711328302
#   (0, 5592661)	-0.015712685556851094
#   (0, 5592662)	0.004005693439832244
#   (0, 5592665)	-0.002501989654314243
#   (0, 5592666)	-0.013165161030087014
#   (0, 5592677)	-0.006570194587953859
#   (0, 5592678)	-0.008638163228606374
#   (0, 5592681)	-0.010117031372320704
#   (0, 5592682)	0.012741686712099838
#   (0, 5592725)	0.011226931763042717
#   :	:
#   (0, 11184490)	0.004498692797457947
#   (0, 11184533)	0.037138063678283505
#   (0, 11184534)	0.004997884820825978
#   (0, 11184537)	0.005609522028966074
#   (0, 11184538)	-0.0032703416322529856
#   (0, 11184549)	0.00036145660802447947
#   (0, 11184550)	-0.004055715445321006
#   (0, 11184553)	-0.007181494585729485
#   (0, 11184554)	-0.021955496476691337
#   (0, 11184725)	0.006563613396898402
#   (0, 11184726)	0.012710753551230003
#   (0, 11184729)	0.009444334164690652
#   (0, 11184730)	-0.017083867347982673
#   (0, 11184741)	0.001145681714070643
#   (0, 11184742)	-0.005446772015578745
#   (0, 11184745)	0.020524464781905644
#   (0, 11184746)	-0.0030569748012595497
#   (0, 11184789)	0.0002066366119244938
#   (0, 11184790)	0.02914517790837405
#   (0, 11184793)	-0.0051560110819075895
#   (0, 11184794)	-0.010525605870457855
#   (0, 11184805)	-0.014505773962148956
#   (0, 11184806)	0.0017993516716503062
#   (0, 11184809)	0.00775899067504589
#   (0, 11184810)	0.010907287268924875
# num non-zero elements = 4096
# Total number of particles = 11.99999999999999
# Total <Sz> = -0.12783663747693844
# Total <S^2> = 9.39475369329993
# Total S = 2.605600375660064
# Local moment squared <m^2> = 12.523276236124483
# Eigenvalues = [-12. -12.]




# 12 sites: 
# (1, 4, 4)
# (4, 4, 16)
# (16, 4, 64)
# (64, 4, 256)
# (256, 4, 770)
# (770, 4, 1129)
# (1129, 4, 776)
# (776, 4, 256)
# (256, 4, 64)
# (64, 4, 16)
# (16, 4, 4)
# (4, 4, 1)
# Ground state energy from exact diagonalization = -15.593764921914902
# num non-zero elements = 852458
# Total number of particles = 11.999999999999533
# Total <Sz> = 0.0
# Total <S^2> = 4.3031489181200165e-18
# Total S = 0.0
# Local moment squared <m^2> = 0.0
# Eigenvalues = [-15.59376492 -15.27117938]
# hopping_strength = 1
# onsite_interaction = 2
# chemical_potential = 0.0
# Build Hami: 12 min; diagonalize: 4 min; RAM maybe up to 22 GB